In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

# Drift-Diffusion Model Exploration

The exercise asks that we repeatedly generate response time distributions from a DDM while manipulating one parameter at a time.

In [2]:
def simulate_diffusion(v, a, beta, tau, dt=1e-3, scale=1.0, max_time=10., rng=None):
    """
    Simulates one realization of the diffusion process given
    a set of parameters and a step size `dt`.

    Parameters:
    -----------
    v        : float
        The drift rate (rate of information uptake)
    a        : float
        The boundary separation (decision threshold).
    beta     : float in [0, 1]
        Relative starting point (prior option preferences)
    tau      : float
        Non-decision time (additive constant)
    dt       : float, optional (default: 1e-3 = 0.001)
        The step size for the Euler algorithm.
    scale    : float, optional (default: 1.0)
        The scale (sqrt(var)) of the Wiener process. Not considered
        a parameter and typically fixed to either 1.0 or 0.1.
    max_time : float, optional (default: .10)
        The maximum number of seconds before forced termination.
    rng      : np.random.Generator or None, optional (default: None)
        A random number generator with locally set seed or None
        If None provided, a new generator will be spawned within the function.
        
    Returns:
    --------
    (x, c) - a tuple of response time (y - float) and a 
        binary decision (c - int) 
    """

    # Inits (process starts at relative starting point)
    y = beta * a
    num_steps = tau
    const = scale*np.sqrt(dt)
    if rng is None:
        rng = np.random.default_rng()

    # Loop through process and check boundary conditions
    while (y <= a and y >= 0) and num_steps <= max_time:

        # Perform diffusion equation
        z = rng.normal()
        y += v*dt + const*z

        # Increment step counter
        num_steps += dt

    if y >= a:
        c = 1.
    else:
        c = 0.
    return (round(num_steps, 4), c)

In [19]:
def simulate_diffusion_n(num_sims, v, a, beta, tau, dt=1e-3, scale=1.0, max_time=10., rng=None):
    """Simulates multiple realizations using simulate_diffusion"""

    # Inits
    data = np.zeros((num_sims, 2))
    if rng is None:
        rng = np.random.default_rng()
    
    # Create data set
    for n in range(num_sims):
        data[n, :] = simulate_diffusion(v, a, beta, tau, dt, scale, max_time, rng)
    return data


def visualize_data(data, figsize=(10, 5)):
    """Helper function to visualize a simple response time data set."""

    f, ax = plt.subplots(1, figsize=figsize)

    # Barplot of categorical responses
    response, frequency = np.unique(data[:, 1], return_counts=True)
    sns.barplot(x=response.astype(np.int32), y=frequency, ax=ax, alpha=0.8, color='#00AA00')

    # Labels and embelishments
    ax.set_xlabel('Response', fontsize=16)
    ax.set_ylabel('Frequency', fontsize=16)
    sns.despine(ax=ax)
    ax.grid(alpha=0.1, color='black')
        
    f.suptitle('Data Summary', fontsize=18)

    f.tight_layout()

Each parameter will be varied 25 times, acquiring 2000 observations each. The first will be drift rate.

In [50]:
# Initialize variables to save statistics
means = np.zeros((25, 2))
sds = np.zeros((25, 2))

# Simulate 25 realizations with different drift values (0.5-1.5)

drift = np.linspace(0.5, 1.5, 25)

for i in range(25):
    
    # Get new drift value
    v = drift[i]
    
    # Set ground-truth parameters
    parameters = {
        'v': v,
        'a': 0.8,
        'beta': 0.6,
        'tau': 0.4
    }
    
    # Gather 2000 realizations
    data = simulate_diffusion_n(num_sims=2000, **parameters)
    
    # Check
    #visualize_data(data)
    
    # Save statistics for each response time
    means[i] = np.mean(data, axis=0)
    sds[i] = np.std(data, axis=0)
    
#means
#sds

Next, 25 iterations of alpha will be calculated.

In [56]:
# Initialize variables to save statistics
means = np.zeros((25, 2))
sds = np.zeros((25, 2))

# Simulate 25 realizations with different boundary sep values (0.4-1.2)

alpha = np.linspace(0.4, 1.2, 25)

for i in range(25):
    
    # Get new a value
    a = alpha[i]
    
    # Set ground-truth parameters
    parameters = {
        'v': 1.5,
        'a': a,
        'beta': 0.6,
        'tau': 0.4
    }
    
    # Gather 2000 realizations
    data = simulate_diffusion_n(num_sims=2000, **parameters)
    
    # Check
    #visualize_data(data)
    
    # Save statistics for each response time
    means[i] = np.mean(data, axis=0)
    sds[i] = np.std(data, axis=0)
    
#means
#sds

Next, 25 realizations of beta.

In [61]:
# Initialize variables to save statistics
means = np.zeros((25, 2))
sds = np.zeros((25, 2))

# Simulate 25 realizations with different beta values (0.2-0.8)

beta = np.linspace(0.2, 0.8, 25)

for i in range(25):
    
    # Get new drift value
    b = beta[i]
    
    # Set ground-truth parameters
    parameters = {
        'v': 1.5,
        'a': 0.8,
        'beta': b,
        'tau': 0.4
    }
    
    # Gather 2000 realizations
    data = simulate_diffusion_n(num_sims=2000, **parameters)
    
    # Check
    #visualize_data(data)
    
    # Save statistics for each response time
    means[i] = np.mean(data, axis=0)
    sds[i] = np.std(data, axis=0)
    
#means
#sds

array([[0.13435766, 0.49792545],
       [0.12575541, 0.499996  ],
       [0.13522643, 0.49967589],
       [0.12958242, 0.49764219],
       [0.1265161 , 0.48513812],
       [0.12615409, 0.48501314],
       [0.13152827, 0.47617618],
       [0.12155962, 0.47851332],
       [0.13406697, 0.45781983],
       [0.13019048, 0.45759999],
       [0.12818877, 0.44421813],
       [0.12614997, 0.43243381],
       [0.12446945, 0.41322845],
       [0.12276095, 0.39269549],
       [0.1248075 , 0.38870265],
       [0.12091336, 0.36845081],
       [0.11934267, 0.36660606],
       [0.12894184, 0.35260424],
       [0.11975384, 0.35007285],
       [0.11584888, 0.31163921],
       [0.11583311, 0.29252308],
       [0.11392508, 0.30264831],
       [0.11508355, 0.26895678],
       [0.10798648, 0.26973876],
       [0.0999923 , 0.24024519]])

Finally, 25 realizations of tau.

In [74]:
# Initialize variables to save statistics
means = np.zeros((25, 2))
sds = np.zeros((25, 2))

# Simulate 25 realizations with different tau values (0.2-0.8)

tau = np.linspace(0, 0.8, 25)

for i in range(25):
    
    # Get new drift value
    t = tau[i]
    
    # Set ground-truth parameters
    parameters = {
        'v': 1.5,
        'a': 0.8,
        'beta': 0.6,
        'tau': t
    }
    
    # Gather 2000 realizations
    data = simulate_diffusion_n(num_sims=2000, **parameters)
    
    # Check
    #visualize_data(data)
    
    # Save statistics for each response time
    #means[i] = np.mean(data, axis=0)
    sds[i] = np.std(data, axis=0)
    
#means
#sds

array([[0.11871932, 0.36890886],
       [0.12136192, 0.36982124],
       [0.12010583, 0.36094875],
       [0.11793509, 0.35707142],
       [0.12357589, 0.3803942 ],
       [0.11819624, 0.37208064],
       [0.11988836, 0.36427153],
       [0.12170736, 0.3756328 ],
       [0.12704737, 0.35950487],
       [0.12258416, 0.35756083],
       [0.1174986 , 0.36332871],
       [0.11867344, 0.37910948],
       [0.12047799, 0.35902089],
       [0.12352108, 0.37607147],
       [0.12084209, 0.36285534],
       [0.11971986, 0.36799151],
       [0.12670258, 0.35410592],
       [0.12054395, 0.37252886],
       [0.12814783, 0.35608847],
       [0.12298863, 0.35756083],
       [0.13045471, 0.35756083],
       [0.12912918, 0.37694529],
       [0.12509993, 0.35853556],
       [0.12320243, 0.37297587],
       [0.1177452 , 0.35853556]])